In [ ]:
# Init
import os
import sys

module_path = os.path.abspath(os.path.join("../src/simulicronalpha/"))
if module_path not in sys.path:
    sys.path.append(module_path)

# Imports
import numpy as np
import pandas as pd
import warnings
import multiprocessing
np.set_printoptions(suppress=True)
from numpy import concatenate as c
from numpy import cumsum
import random
%load_ext line_profiler

In [ ]:
from generateSim import (generatePopulation, generateGenome)
from stats import stats
from regulation import regulation
from checkCopyNumber import checkCopyNumber
from fitness import calculateFitness
from transposition import transposition
from recombination import recombination

In [ ]:
from popSim import runBatch, runSim

In [ ]:
gen,piset,piIndice, rates = generateGenome(
    numberOfInsertionSites=10000,
    numberOfChromosomes=6,
    baseRecombinationRate=0.1,
    baseSelection=0,
    baseTau=1,
)
pop, tr , TEset= generatePopulation(
    gen,
    piIndice,
    NumberOfIndividual=1000,
    NumberOfTransposonTypes=2,
    NumberOfInsertionsPerType=[1,1],
    FrequencyOfInsertions=[0.5,0.5],
    ExcisionRates = [0.2,0.5],
    RepairRates=[1,1],
    InsertionRates=[1,1],
)

In [ ]:
runSim(gen,pop,tr,TEset,2,101,rates,piset)

In [5]:
%lprun -f runSim runSim(gen,pop,tr,TEset,1,10000,rates,piset)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
*** KeyboardInterrupt exception caught in code being profiled.

Timer unit: 1e-06 s

Total time: 242.127 s
File: /home/siddharth/Documents/Projects/Simulicron/src/simulicronalpha/popSim.py
Function: runSim at line 26

Line #      Hits         Time  Per Hit   % Time  Line Contents
    26                                           def runSim(
    27                                               genomeMatrix,
    28                                               populationMatrix,
    29                                               transposonMatrix,
    30                                               TEset,
    31                                               NumberOfTransposonInsertions,
    32                                               generations,
    33                                               genMap,
    34                                               piSet,
    35                                           ):
    36                                               # ------------------#
    37                                               # l

In [ ]:
%lprun -f recombination recombination(rates=rates, transposonMatrix=tr, v1=[52], v2=[65])

In [ ]:
%lprun -f runBatch runBatch(numberOfSimulations=1,baseSelection=0,baseInsertionProb=1,numberOfInsertionSites=10000,numberOfChromosomes=6,baseRecombinationRate=0.01,baseTau=1,numberOfPiRNA=6,piPercentage=3,enablePiRecombination=False,NumberOfIndividual=1000,NumberOfTransposonTypes=1,NumberOfInsertionsPerType=[1],FrequencyOfInsertions=[1.0],ExcisionRates=[1.0],RepairRates=[1],InsertionRates=[1],HardyWeinberg=False,NumberOfGenerations=10,numberOfThreads=1,)

In [ ]:
np.set_printoptions(suppress=True,)

In [ ]:
%lprun -f transposition transposition(transposonMatrix=tr, genomeMatrix=gen, NumberOfTransposonInsertions=1, TEset=TEset, v1=[52,101,321,433,12,23,234,123,657,29], v2=[65,54,22,121,23])

In [ ]:
def checkCopyNumber(populationMatrix, TEset, transposonMatrix):
    # The local flatten lambda
    flatten = lambda *n: (
        e for a in n for e in (flatten(*a) if isinstance(a, (tuple, list)) else (a,))
    )
    # Convert the list to flat list and remove 0
    # if present.
    # Also create a dictionary counting 
    # each family type
    TEfamilyCount = {k: [] for k in range(1, len(TEset.keys()) + 1)}
    TEfamilyVar = {k: [] for k in range(1, len(TEset.keys()) + 1)}
    TETotal = []
    for i in range(len(populationMatrix)):
        v1 = populationMatrix[i, 0] if type(populationMatrix[i, 0]) is list else []
        v2 = populationMatrix[i, 1] if type(populationMatrix[i, 1]) is list else []
        if (v1 + v2 == []):
            TETotal.append(0)
            for k in TEset.keys():
                TEfamilyCount[k].append(0)
        else:
            # Check the occupancy for each family
            TEfamilies = transposonMatrix[v1+v2, 0].tolist()
            TETotal.append(len(TEfamilies))
            for k in TEset.keys():
                TEfamilyCount[k].append(TEfamilies.count(k))
    # Calculate variance per family and sum the count
    for k in TEset.keys():
        TEfamilyVar[k].append(np.var(TEfamilyCount[k]))
        TEfamilyCount[k] = sum(TEfamilyCount[k])
    return sum(TETotal), np.var(TETotal), TEfamilyCount, TEfamilyVar

In [ ]:
TEfamilyCount = {k: [] for k in range(1, len(TEset.keys()) + 1)}

In [ ]:
x, y, a, b = checkCopyNumber(pop, TEset, tr)

In [ ]:
a

In [ ]:
TEfamilyCount

In [ ]:
pop

In [ ]:
tr

In [ ]:
z = [3,4,6,74,4,3]

In [ ]:
z.count(7)

In [ ]:
def runSim(
    genomeMatrix,
    populationMatrix,
    transposonMatrix,
    TEset,
    NumberOfTransposonInsertions,
    generations,
    genMap,
    piSet,
):
    # ------------------#
    # lambda/macros
    flatten = lambda *n: (
        e
        for a in n
        for e in (
            flatten(*a) if isinstance(a, (tuple, list)) else (a,)
        )
    )
    # ------------------#
    # ------------------#
    # for storing transposons which are fixed
    fixedTE = []
    # for storing transposons which are not fixed
    unfixedTE = []
    # for storing transposons which are lost
    lostTE = []
    # For storing the average copy number per generation
    averageCopyNumber = []
    # For storing the copy number variance per generation
    varianceCopyNumber = []
    # Per family statistics
    TEfamilyCountArr = []
    TEfamilyVarArr = []
    # Create a insertionSiteSepecific array
    insertionSiteFrequencyArray = np.zeros(
        (len(genomeMatrix), NumberOfTransposonInsertions,)
    )
    transposonMatrixCopy = transposonMatrix
    populationMatrixCopy = populationMatrix

    # Create a fixed size array for storing transposon information
    # Also add a "counter" - with a large fixed array, it is not
    # possible to use len(transposons) to find the last filled index
    transposonMatrixCopy = np.append(
        transposonMatrix,
        np.zeros((10000000, 6), dtype=object),
        axis=0,
    )
    numberOfTranspositionEvents = len(transposonMatrix)

    # Calculate piRNA coordinates
    piCoord = []
    for i in piSet.values():
        piCoord = piCoord + (list(range(i[0], i[1])))

    # Calculate the CN and CNV for generation 0
    (
        copyNumber,
        varianceNumber,
        TEfamilyCount,
        TEfamilyVar,
    ) = checkCopyNumber(
        populationMatrixCopy, TEset, transposonMatrixCopy
    )
    averageCopyNumber.append(copyNumber)
    varianceCopyNumber.append(varianceNumber)
    TEfamilyCountArr.append(TEfamilyCount)
    TEfamilyVarArr.append(TEfamilyVar)

    # Driver loop
    for i in range(generations):
        # Start the clock
        # timeStart = time.time()
        print(i)
        populationV1 = []
        populationV2 = []
        populationFit = []
        for k in list(range(populationMatrixCopy.shape[0])):
            fitness = list(populationMatrixCopy[0:, 2])

            p1, p2 = random.choices(
                list(range(populationMatrixCopy.shape[0])),
                weights=fitness,
                k=2,
            )

            # Since recombination function only accepts arrays,
            # checking and forcing type conversion as needed
            # for the respective alleles
            if (
                populationMatrixCopy[p1, 0] == 0
                and populationMatrixCopy[p1, 1] == 0
            ):
                v1 = 0
            else:
                cP1V1 = populationMatrixCopy[p1, 0]
                cP1V2 = populationMatrixCopy[p1, 1]
                if isinstance(cP1V1, list):
                    cP1V1 = np.asarray(cP1V1)
                else:
                    cP1V1 = np.asarray([cP1V1])
                if isinstance(cP1V2, list):
                    cP1V2 = np.asarray(cP1V2)
                else:
                    cP1V2 = np.asarray([cP1V2])

                v1 = recombination(
                    genMap, transposonMatrixCopy, v1=cP1V1, v2=cP1V2,
                )

            if (
                populationMatrixCopy[p2, 0] == 0
                and populationMatrixCopy[p2, 1] == 0
            ):
                v2 = 0
            else:
                cP2V1 = populationMatrixCopy[p2, 0]
                cP2V2 = populationMatrixCopy[p2, 1]
                if isinstance(cP2V1, list):
                    cP2V1 = np.asarray(cP2V1)
                else:
                    cP2V1 = np.asarray([cP2V1])
                if isinstance(cP2V2, list):
                    cP2V2 = np.asarray(cP2V2)
                else:
                    cP2V2 = np.asarray([cP2V2])

                v2 = recombination(
                    genMap, transposonMatrixCopy, v1=cP2V1, v2=cP2V2,
                )

            if v1 == 0 and v2 == 0:
                indFitness = 1

            else:
                (
                    v1,
                    v2,
                    transposonMatrixCopy,
                    TEset,
                    numberOfTranspositionEvents,
                ) = transposition(
                    transposonMatrix=transposonMatrixCopy,
                    genomeMatrix=genomeMatrix,
                    NumberOfTransposonInsertions=NumberOfTransposonInsertions,
                    TEset=TEset,
                    piCoord=piCoord,
                    numberOfTranspositionEvents=numberOfTranspositionEvents,
                    v1=v1,
                    v2=v2,
                )
                indFitness = calculateFitness(
                    transposonMatrixCopy, v1, v2
                )

            populationV1.append(v1)
            populationV2.append(v2)
            populationFit.append(indFitness)
        # Return i+2 since i start at 0 = generation 1
        # and the condition check happens at generation
        # n-1, hence i + 1 + 1

        # Check if there are no transposons left
        if False:
            # Make the family information "flat"
            # https://stackoverflow.com/questions/5946236/how-to-merge-multiple-dicts-with-same-key
            dict1 = {}
            dict2 = {}
            for k in TEset.keys():
                dict1[k] = tuple(dict1[k] for dict1 in TEfamilyCountArr)
                dict2[k] = tuple(dict2[k] for dict2 in TEfamilyCountArr)
            TEfamilyCountArrRes = dict1
            TEfamilyVarArrRes = dict2
            return {
                "State": "LOSS",
                "Generatrion": i + 2,
                "NTE": numberOfTranspositionEvents,
                "AvgCopyNum": averageCopyNumber,
                "CopyNumVar": varianceCopyNumber,
                "TEfamilyCN": TEfamilyCountArrRes,
                "TEfamilyVR": TEfamilyVarArrRes,
            }
        else:
            pass
        # Major bug in numpy - forced to use pandas
        # Refer to the question
        # https://stackoverflow.com/questions/60210897
        # populationMatrixCopy = pd.DataFrame(
        #    [populationV1, populationV2, populationFit]
        # ).T.to_numpy()
        populationMatrixCopy= np.array([populationV1,
            populationV2,
            populationFit,], dtype="object").T
        # (
        #     compyNumber,
        #     varianceNumber,
        #     insertionSiteFrequencyArray,
        # ) = statistics(
        #     populationMatrixCopy,
        #     transposonMatrixCopy,
        #     TEset,
        #     insertionSiteFrequencyArray,
        # )
        (
            copyNumber,
            varianceNumber,
            TEfamilyCount,
            TEfamilyVar,
        ) = checkCopyNumber(
            populationMatrixCopy, TEset, transposonMatrixCopy
        )
        averageCopyNumber.append(copyNumber)
        varianceCopyNumber.append(varianceNumber)
        TEfamilyCountArr.append(TEfamilyCount)
        TEfamilyVarArr.append(TEfamilyVar)
        # Stop the timer
        # timeStop = time.time()
        # Terminate the loop if it runs for more than 
        # 8 seconds! 
        # if (timeStop-timeStart) > 10.0:
        #    return {
        #        "State": "TIMEXC",
        #        "Generatrion": i + 2,
        #        "NTE": numberOfTranspositionEvents,
        #        "AvgCopyNum": averageCopyNumber,
        #        "CopyNumVar": varianceCopyNumber,
        #        "TEfamilyCN": TEfamilyCountArr,
        #        "TEfamilyVR": TEfamilyVarArr,
        #    }

    # Quit simulation if there in a transient state
    # i.e. no loss
    dict1 = {}
    dict2 = {}
    for k in TEset.keys():
        dict1[k] = tuple(dict1[k] for dict1 in TEfamilyCountArr)
        dict2[k] = tuple(dict2[k] for dict2 in TEfamilyCountArr)
    TEfamilyCountArrRes = dict1
    TEfamilyVarArrRes = dict2
    return {
            "State": "FLUX",
            "Generatrion": i + 2,
            "NTE": numberOfTranspositionEvents,
            "AvgCopyNum": averageCopyNumber,
            "CopyNumVar": varianceCopyNumber,
            "TEfamilyCN": TEfamilyCountArrRes,
            "TEfamilyVR": TEfamilyVarArrRes,
    }

In [ ]:
checkCopyNumber(pop)

In [ ]:
gen